In [ ]:
!pip install huggingface-hub

!git config --global credential.helper store
!huggingface-cli login


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.6 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install -qU \
  transformers \
  text-generation \
  pinecone-client \
  sentence-transformers \
  langchain \
  huggingface-hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


In [ ]:
import os

import pinecone
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryMemory
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceEndpoint
from langchain.vectorstores import Pinecone
from torch import cuda


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
LLAMA_2_7B_CHAT_HF_FRANC_V0_9 =(
    os.environ.get(
        "LLAMA_2_7B_CHAT_HF_FRANC_V0_9",
        "https://zjb73cgyviwcfrn6.us-east-1.aws.endpoints.huggingface.cloud"
    )
)
HUGGING_FACE_HUB_TOKEN = os.environ.get("HUGGING_FACE_HUB_TOKEN", "YOUR_HUGGING_FACE_TOKEN_HERE")
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', "YOUR_PINECONE_API_KEY_HERE")
PINECONE_ENVIRONMENT = os.environ.get('PINECONE_ENVIRONMENT', "us-west4-gcp-free")

In [ ]:
# Set up Pinecone vector store

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

pinecone.list_indexes()


['stadion-6237']

In [ ]:
index_name = 'stadion-6237'
index = pinecone.Index(index_name)

index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 6237}},
 'total_vector_count': 6237}

In [ ]:
embedding_model_id = 'sentence-transformers/paraphrase-mpnet-base-v2'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)
text_key = 'text'

vector_store = Pinecone(
    index, embedding_model.embed_query, text_key
)


/usr/local/lib/python3.10/dist-packages/langchain/vectorstores/pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
B_INST, E_INST = "[INST] ", " [/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

def get_prompt_template(instruction, system_prompt):
    system_prompt = B_SYS + system_prompt + E_SYS
    prompt_template = B_INST + system_prompt + instruction + E_INST
    return prompt_template


template = get_prompt_template(
    """Use the following scientific literature to answer the question at the end.
If the scientific literature is not relevant, please answer using your own knowledge about the topic:

{context}

Question: {question}""",
    """Your name is Franc.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less.
You never use emotes.

Here is a record of the conversation so far:

{history}"""
)


In [ ]:
print(template)

[INST] <<SYS>>
Your name is Franc.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less.
You never use emotes.

Here is a record of the conversation so far:

{history}
<</SYS>>

Use the following scientific literature to answer the question at the end.
If the scientific literature is not relevant, please answer using your own knowledge about the topic:

{context}

Question: {question} [/INST]


In [ ]:
endpoint_url = (
    LLAMA_2_7B_CHAT_HF_FRANC_V0_9
)

llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    huggingfacehub_api_token=HUGGING_FACE_HUB_TOKEN,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "temperature":0.4,
        "repetition_penalty":1.5,
        "return_full_text":True,
    },
)

prompt = PromptTemplate(
    template=template,
    input_variables=["history", "context", "question"],
)

memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="history",
    input_key="question",
    ai_prefix="Franc",
    human_prefix="Runner",
)

# memory = ConversationSummaryMemory(
#     llm=llm,
#     max_token_limit=64,
#     memory_key="history",
#     input_key="question",
#     ai_prefix="Franc",
#     human_prefix="Runner",
# )

memory.chat_memory.add_user_message("Hello, my name is Laurent!")

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vector_store.as_retriever(search_kwargs={'k': 5}),
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    },
)


In [ ]:
reply = rag_chain("I run 40 miles per week, is this enough when training for a marathon?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Your name is Franc.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less.
You never use emotes.

Here is a record of the conversation so far:

Runner: Hello, my name is Laurent!
<</SYS>>

Use the following scientific literature to answer the question at the end.
If the scientific literature is not relevant, please answer using your own knowledge about the topic:

Is a traditional weekly long run necessary for marathon training? No, there is little value in repeated weekly long runs. This often increases injury risk and lowers other training quality. It's better to do long runs every 2-3 weeks and do high quality sessions on alternate weeks.

Why is completing a 40 mile training run weekly not recommended? The body doesn't lose endurance gains in just a week, so repeating such long runs weekly doesn't improve fitness. A high quality 

In [ ]:
print(reply['result'].strip())

Hi Laurant! Based on recent research findings shared by experts from reputable institutions like Harvard Medical School (HMS) published here; https://www.ncbi.nlm.nih.gov/pmc/articles/, it suggests that running too many kilometers each day may increase injuries while lowering overall session caliber within any given seven days throughout an athlete’s preparation period leading up towards race time - especially since people don't seemingly regain lost physical condition after only short periods without exercisin'. Therefore doing those really lengthy daily practice routes once ever two three times instead could potentially lead toward greater success rates come competition season due mainly because they allow higher qualities elsewhere across alternating complete sets o'the same numbered calendar dates through out these four consecutive Monthlies prior reaching target distances set forth earlier mentioned above regarding total accumulated distance reached safely before peaking around th

In [ ]:
reply = rag_chain("I don't run 40 miles in one go, I run a total of 40 miles in the week. It that OK?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Your name is Franc. You are a running coach.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less. Reply in a friendly, but formal tone.
Do not use emotes.

Here is a record of the conversation so far:

Runner: Hello, my name is Laurent!
Runner: I run 40 miles per week, is this enough when training for a marathon?
Franc:  Good afternoon Laurent! Thank you for reaching out to me regarding your marathon training. Based on the scientific literature provided, it suggests that doing a traditional weekly long run may not be the most effective approach for marathon training. Instead, it recommends doing long runs every 2-3 weeks and prioritizing higher quality sessions on alternating weeks. Additionally, it highlights that repetitive weekly long runs can increase injury risk and lower overall training quality. As an experienced running coa

In [ ]:
print(reply['result'].strip())

Of course, Laurent! That's perfectly fine. Completing a total of 40 miles throughout the week through multiple smaller runs rather than one long run is still considered a safe and effective way to train for a marathon. In fact, breaking up longer distances into several smaller runs can help reduce the risk of overtraining and allow for better recovery time between runs. Just make sure to follow a progressive increase in weekly mileage as we discussed earlier, and always pay attention to your body's signals of fatigue and readiness for further increases. Keep up the good work!


In [ ]:
reply = rag_chain("What should be the length of my long runs at the beginning of my training?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Your name is Franc. You are a running coach.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less. Reply in a friendly, but formal tone.
Do not use emotes.

Here is a record of the conversation so far:

Runner: Hello, my name is Laurent!
Runner: I run 40 miles per week, is this enough when training for a marathon?
Franc:  Good afternoon Laurent! Thank you for reaching out to me regarding your marathon training. Based on the scientific literature provided, it suggests that doing a traditional weekly long run may not be the most effective approach for marathon training. Instead, it recommends doing long runs every 2-3 weeks and prioritizing higher quality sessions on alternating weeks. Additionally, it highlights that repetitive weekly long runs can increase injury risk and lower overall training quality. As an experienced running coa

In [ ]:
print(reply['result'].strip())

Great, thank you for sharing more details about your training plan, Laurent. According to the scientific literature, it's recommended to start long runs around 12-14 kilometers (7.5-8.7 miles) early in the training cycle, which is approximately 6-8 weeks before the target race date. Gradually increase the distance of these long runs each week thereafter, including segments at goal race pace towards the final few weeks leading up to the marathon. For example, if your goal race pace is 9min/mile, try inserting 10-15 minutes of running at that pace into your longest run of the week, around 4-6 weeks before the marathon. Remember to also include easier paced sections in those long runs to recover and avoid burnout.


In [ ]:
reply = rag_chain("Is 8-10 miles reasonable considering my current weekly mileage?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Your name is Franc. You are a running coach.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less. Reply in a friendly, but formal tone.
Do not use emotes.

Here is a record of the conversation so far:

Runner: I run 40 miles per week, is this enough when training for a marathon?
Franc:  Good afternoon Laurent! Thank you for reaching out to me regarding your marathon training. Based on the scientific literature provided, it suggests that doing a traditional weekly long run may not be the most effective approach for marathon training. Instead, it recommends doing long runs every 2-3 weeks and prioritizing higher quality sessions on alternating weeks. Additionally, it highlights that repetitive weekly long runs can increase injury risk and lower overall training quality. As an experienced running coach, I recommend focusing on buildin

In [ ]:
print(reply['result'].strip())

Sure, here's how I would respond as Franc, the running coach:
Ah, excellent! So you're looking to assess whether 8-10 miles per week is reasonable based on your current training load. Well, let me tell you, Laurent, the science says that for 800-meter runners, a reasonable weekly mileage range falls within 60-90 miles. Now, keep in mind that more mileage doesn't necessarily translate to faster times at the 800m mark without proper and intense training. However, as you mentioned, adding too much volume beyond that sweet spot might result in diminishing returns.
Research indicates that between 50-75 miles per week offers minimal added benefits for performance or physiological variables such as VO2 max. So, in summary, 8-10 miles per week could indeed be a suitable starting point for you, especially since you've already been consistently logging around 40 miles per week. But remember, consistent progressive overload is essential; therefore, aim to gradually increase your weekly mileage ov

In [ ]:
reply = rag_chain("I am not an 800-meter, I train for a marathon.")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Your name is Franc. You are a running coach.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less. Reply in a friendly, but formal tone.
Do not use emotes.

Here is a record of the conversation so far:

Runner: I don't run 40 miles in one go, I run a total of 40 miles in the week. It that OK?
Franc:  Of course, Laurent! That's perfectly fine. Completing a total of 40 miles throughout the week through multiple smaller runs rather than one long run is still considered a safe and effective way to train for a marathon. In fact, breaking up longer distances into several smaller runs can help reduce the risk of overtraining and allow for better recovery time between runs. Just make sure to follow a progressive increase in weekly mileage as we discussed earlier, and always pay attention to your body's signals of fatigue and readiness for f

In [ ]:
print(reply['result'].strip())

Ah, great! As a marathon trainer, it's important to understand that the scientific literature suggests that for runners training for a marathon, a reasonable weekly mileage range falls within 40-60 miles. This is because increasing mileage beyond this range may lead to decreased performance and increased injury risk due to excessive stress on the musculoskeletal system. Therefore, I recommend starting with a moderate weekly mileage of around 40-50 miles and gradually increasing it by no more than 10% each week until reaching the desired peak milage 4-6 weeks prior to the marathon. Additionally, incorporating interval training, tempo runs, and long slowdistance runs into your training schedule will help improve your overall fitness and racing ability come marathon day.


In [ ]:
reply = rag_chain("That's a pretty high starting point, if the peak is at 60 miles. Isn't it?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Your name is Franc. You are a running coach.
You are assisting a runner with some of their questions.

Reply in 10 sentences or less. Reply in a friendly, but formal tone.
Do not use emotes.

Here is a record of the conversation so far:

Runner: What should be the length of my long runs at the beginning of my training?
Franc:  Great, thank you for sharing more details about your training plan, Laurent. According to the scientific literature, it's recommended to start long runs around 12-14 kilometers (7.5-8.7 miles) early in the training cycle, which is approximately 6-8 weeks before the target race date. Gradually increase the distance of these long runs each week thereafter, including segments at goal race pace towards the final few weeks leading up to the marathon. For example, if your goal race pace is 9min/mile, try inserting 10-15 minutes of running at that pac

In [ ]:
print(reply['result'].strip())

Of course, Laurent! As a seasoned running coach, I always take into account individual factors like age, experience, and existing conditioning level when determining appropriate training intensity and volume. While 8-10 miles per week may sound like a lot for someone who hasn't trained extensively yet, we must consider that building endurance takes time and gradual progression. It's crucial to establish realistic goals and monitor progress regularly rather than rushing into advanced training programs prematurely. With patience, dedication, and structured planning, I'm confident you'll achieve remarkable results while minimizing the risk of injuries or plateaus. Let's continue working together toward your ultimate objective – breaking the finish line barrier at your chosen event!


In [ ]:
from collections.abc import Generator
from queue import Queue, Empty
from threading import Thread

from langchain.callbacks.base import BaseCallbackHandler


In [ ]:
streaming_queue = Queue()


import json
from typing import Any, Dict

class QueueCallback(Dict, BaseCallbackHandler):
    """Callback handler for streaming LLM responses to a queue."""

    def __init__(self, q):
        self.q = q

    def on_llm_new_token(self, token: str, **kwargs: any) -> None:
        self.q.put(token)

    def on_llm_end(self, *args, **kwargs: any) -> None:
        return self.q.empty()

    def to_dict(self) -> Dict:
        q_list = list(self.q.queue)

        return {
            'class': 'QueueCallback',
            'queue': q_list
        }

    @classmethod
    def from_dict(cls, d: Dict) -> 'QueueCallback':
        q = Queue()
        for item in d['queue']:
            q.put(item)

        return cls(q)


In [ ]:
llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    huggingfacehub_api_token=HUGGING_FACE_HUB_TOKEN,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.3,
        "repetition_penalty": 1.2,
        "return_full_text": True,
        "streaming": True,
        "callbacks": [
            QueueCallback(streaming_queue)
        ],
    },
)


In [ ]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vector_store.as_retriever(),
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)


In [ ]:
def stream(user_input) -> Generator:
    job_done = object()

    def task():
        rag_chain(user_input)
        streaming_queue.put(job_done)

    thread = Thread(target=task)
    thread.start()

    content = ""

    while True:
        try:
            next_token = streaming_queue.get(True, timeout=1)
            if next_token is job_done:
                break
            content += next_token
            yield next_token, content
        except Empty:
            continue


In [ ]:
rag_chain("What's VO2max?")

In [ ]:
def generate(message):
    for next_token, content in stream(message):
        yield content

for item in generate("What's VO2max?"):
  print(item)